<a href="https://colab.research.google.com/github/sumanyurosha/tensorflow-specialization/blob/master/Course3/Week2/Lesson2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json


--2020-09-30 18:58:23--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.195.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.02s   

2020-09-30 18:58:23 (306 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [11]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import json

In [12]:
with open("/tmp/sarcasm.json", 'r') as f:
    sarcasm_dataset = json.load(f)

sentences = []
labels = []

for item in sarcasm_dataset:
    sentences.append(item["headline"])
    labels.append(item["is_sarcastic"])

In [13]:
vocab_size = 10000
oov_token = "<OOV>"
max_len = 100
batch_size = 32
embedding_dim = 16
trunc_type = "post"
padding_type = "post"
training_size = 20000

In [14]:
training_sentences = sentences[:training_size]
training_labels = labels[:training_size]

testing_sentences = sentences[training_size:]
testing_labels = labels[training_size:]

print(len(training_sentences))
print(len(testing_sentences))

20000
6709


In [15]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

In [16]:
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)

testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [17]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(6, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss="binary_crossentropy", 
              optimizer="adam",
              metrics=["accuracy"])

history = model.fit(training_padded, training_labels, epochs=30, validation_data=(testing_padded, testing_labels), verbose=1)

Epoch 1/30
625/625 [==============================] - 3s 4ms/step - loss: 0.6751 - accuracy: 0.5627 - val_loss: 0.6400 - val_accuracy: 0.6399
Epoch 2/30
625/625 [==============================] - 3s 4ms/step - loss: 0.5471 - accuracy: 0.7772 - val_loss: 0.4710 - val_accuracy: 0.8025
Epoch 3/30
625/625 [==============================] - 3s 4ms/step - loss: 0.3922 - accuracy: 0.8544 - val_loss: 0.3897 - val_accuracy: 0.8422
Epoch 4/30
625/625 [==============================] - 3s 5ms/step - loss: 0.3204 - accuracy: 0.8770 - val_loss: 0.3605 - val_accuracy: 0.8520
Epoch 5/30
625/625 [==============================] - 3s 4ms/step - loss: 0.2783 - accuracy: 0.8942 - val_loss: 0.3474 - val_accuracy: 0.8542
Epoch 6/30
625/625 [==============================] - 3s 4ms/step - loss: 0.2472 - accuracy: 0.9043 - val_loss: 0.3424 - val_accuracy: 0.8572
Epoch 7/30
625/625 [==============================] - 3s 4ms/step - loss: 0.2230 - accuracy: 0.9156 - val_loss: 0.3471 - val_accuracy: 0.8499
Epoch 